In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Prepare data

In [27]:
hydraulic = pd.read_csv('HourlyData - DS.csv')

In [181]:
# hydraulic.head()

In [12]:
########################################################################################
# Per "profile.txt" from original data source, 4 conditions are defined as:
# 1: Cooler condition / %:
# 3: close to total failure
# 20: reduced effifiency
# 100: full efficiency

# 2: Valve condition / %:
# 100: optimal switching behavior
# 90: small lag
# 80: severe lag
# 73: close to total failure

# 3: Internal pump leakage:
# 0: no leakage
# 1: weak leakage
# 2: severe leakage

# 4: Hydraulic accumulator / bar:
# 130: optimal pressure
# 115: slightly reduced pressure
# 100: severely reduced pressure
# 90: close to total failure

# The function mark condition flag to 1 for failure and 0 for optimal
#
########################################################################################

# def condition_flag(cols, col_name):
#     Condition = cols[col_name]
    
#     if col_name == 'Cooler condition':
#         if Condition == 3 or Condition == 20:
#             return 1
#         elif Condition == 100:
#             return 0
#         else:
#             return Condition
        
#     elif col_name == 'Valve condition':
#         if Condition == 73 or Condition == 80:
#             return 1
#         elif Condition == 90 or Condition == 100:
#             return 0
#         else:
#             return Condition

#     elif col_name == 'Internal pump leakage':
#         if Condition == 1 or Condition == 2:
#             return 1
#         else:
#             return Condition            # if 0 or others just return as is  

#     elif col_name == 'Hydraulic accumulator':
#         if Condition == 90 or Condition == 100:
#             return 1
#         elif Condition == 115 or Condition == 130:
#             return 0
#         else:
#             return Condition

#     else:
#          return Condition

In [28]:
########################################################################################
# Per "profile.txt" from original data source, 4 conditions are defined as:
# 1: Cooler condition / %:
# 3: close to total failure
# 20: reduced effifiency
# 100: full efficiency

# 2: Valve condition / %:
# 100: optimal switching behavior
# 90: small lag
# 80: severe lag
# 73: close to total failure

# 3: Internal pump leakage:
# 0: no leakage
# 1: weak leakage
# 2: severe leakage

# 4: Hydraulic accumulator / bar:
# 130: optimal pressure
# 115: slightly reduced pressure
# 100: severely reduced pressure
# 90: close to total failure

# The function mark condition flag to 1 for failure and 0 for optimal
#
########################################################################################

def cooler_flag(cols):
    # Re-flag Cooler condition
    condition = cols['Cooler condition']
    
    if condition == 3 or condition == 20:
        return 1
    elif condition == 100:
        return 0
    else:
        return condition

def valve_flag(cols):
    # Re-flag Valve condition
    condition = cols['Valve condition']
    
    if condition == 73 or condition == 80:
        return 1
    elif condition == 90 or condition == 100:
        return 0
    else:
        return condition

def pump_flag(cols):
    # Re-flag Pump condition
    condition = cols['Internal pump leakage']
    
    if condition == 1 or condition == 2:
        return 1
    else:
        return condition     # if 0 or others just return as is

def accumulator_flag(cols):
    # Re-flag accumulator condition
    condition = cols['Hydraulic accumulator']
    
    if condition == 90 or condition == 100:
        return 1
    elif condition == 115 or condition == 130:
        return 0
    else:
        return condition

In [29]:
hydraulic['Cooler condition'] = hydraulic[['Cooler condition']].apply(cooler_flag,axis=1)
hydraulic['Valve condition'] = hydraulic[['Valve condition']].apply(valve_flag,axis=1)
hydraulic['Internal pump leakage'] = hydraulic[['Internal pump leakage']].apply(pump_flag,axis=1)
hydraulic['Hydraulic accumulator'] = hydraulic[['Hydraulic accumulator']].apply(accumulator_flag,axis=1)

In [30]:
hydraulic.head()

,Time,Cooling efficiency,Cooling power,Motor power W,Volume flow l/min 1,Volume flow l/min 2,Pressure bar 1,Pressure bar 2,Pressure bar 3,Pressure bar 4,...,Temperature 2,Temperature 3,Temperature 4,Vibration mm/s,Date,Cooler condition,Valve condition,Internal pump leakage,Hydraulic accumulator,stable flag
0,0,47.202,2.184,2411.6,8.990,10.179,151.47,125.50,2.305,0.0,...,40.961,38.320,30.363,0.604,4/26/2018 0:00,1,0,0,0,1
1,60,29.208,1.414,2409.6,8.919,10.408,151.11,125.06,2.281,0.0,...,41.258,38.680,33.648,0.590,4/26/2018 1:00,1,0,0,0,1
2,120,23.554,1.159,2397.8,9.179,10.392,150.81,125.13,2.227,0.0,...,42.129,39.234,35.113,0.578,4/26/2018 2:00,1,0,0,0,1
3,180,21.540,1.101,2383.8,9.034,10.329,150.48,124.93,2.320,0.0,...,43.039,40.086,36.133,0.565,4/26/2018 3:00,1,0,0,0,1
4,240,20.460,1.086,2372.0,8.729,10.276,150.41,124.72,2.250,0.0,...,44.031,40.934,36.992,0.570,4/26/2018 4:00,1,0,0,0,1


# 1. Target Cooler condition

# Train Test Split 

** Split your data into a training set and a testing set.**

In [31]:
X = hydraulic.drop(['Date', 'Cooler condition', 'Valve condition', 'Internal pump leakage', 'Hydraulic accumulator', 'stable flag'],axis=1)
y = hydraulic['Cooler condition']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Train a Model
  Train a Support Vector Machine Classifier.

In [32]:
svc_model = SVC()

In [33]:
svc_model.fit(X_train,y_train)

SVC()

## Model Evaluation
Now get predictions from the model and create a confusion matrix and a classification report.

In [34]:
predictions = svc_model.predict(X_test)

In [35]:
from sklearn.metrics import classification_report,confusion_matrix

In [36]:
print(confusion_matrix(y_test,predictions))

[[223   2]
 [  0 437]]


In [37]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       225
           1       1.00      1.00      1.00       437

    accuracy                           1.00       662
   macro avg       1.00      1.00      1.00       662
weighted avg       1.00      1.00      1.00       662



# Gridsearch Practice

In [16]:
from sklearn.model_selection import GridSearchCV

#### Create a dictionary called param_grid and fill out some parameters for C and gamma.

In [17]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]} 

** Create a GridSearchCV object and fit it to the training data.**

In [38]:
# grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
# grid.fit(X_train,y_train)

** Now take that grid model and create some predictions using the test set and create classification reports and confusion matrices for them. Were you able to improve?**

In [19]:
grid_predictions = grid.predict(X_test)

In [20]:
print(confusion_matrix(y_test,grid_predictions))

[[  0 233]
 [  0 429]]


In [39]:
# print(classification_report(y_test,grid_predictions))

# 2. Target Valve condition

# Train Test Split

In [40]:
y = hydraulic['Valve condition']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Train a Model

In [41]:
svc_model = SVC()
svc_model.fit(X_train,y_train)

SVC()

# Model Evaluation

In [42]:
predictions = svc_model.predict(X_test)

In [43]:
print(confusion_matrix(y_test,predictions))

[[444   0]
 [218   0]]


In [44]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80       444
           1       0.00      0.00      0.00       218

    accuracy                           0.67       662
   macro avg       0.34      0.50      0.40       662
weighted avg       0.45      0.67      0.54       662



C:\Users\perry\anaconda3\envs\Py310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\perry\anaconda3\envs\Py310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\perry\anaconda3\envs\Py310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Target stable flag

# Train Test Split

In [45]:
y = hydraulic['stable flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Train a Model

In [46]:
svc_model = SVC()
svc_model.fit(X_train,y_train)

SVC()

# Model Evaluation

In [47]:
predictions = svc_model.predict(X_test)

In [48]:
print(confusion_matrix(y_test,predictions))

[[433   5]
 [150  74]]


In [49]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.74      0.99      0.85       438
           1       0.94      0.33      0.49       224

    accuracy                           0.77       662
   macro avg       0.84      0.66      0.67       662
weighted avg       0.81      0.77      0.73       662



# Gridsearch Practice

In [50]:
from sklearn.model_selection import GridSearchCV

#### Create a dictionary called param_grid and fill out some parameters for C and gamma.

In [51]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]} 

** Create a GridSearchCV object and fit it to the training data.**

In [52]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END .....................................C=0.1, gamma=1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.2s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ..................................C=0.1, gamma=0.01; total time=   0.1s
[CV] END ..................................C=0.1

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001]},
             verbose=2)

** Now take that grid model and create some predictions using the test set and create classification reports and confusion matrices for them. Were you able to improve?**

In [53]:
grid_predictions = grid.predict(X_test)

In [54]:
print(confusion_matrix(y_test,grid_predictions))

[[438   0]
 [224   0]]


In [55]:
print(classification_report(y_test,grid_predictions))

              precision    recall  f1-score   support

           0       0.66      1.00      0.80       438
           1       0.00      0.00      0.00       224

    accuracy                           0.66       662
   macro avg       0.33      0.50      0.40       662
weighted avg       0.44      0.66      0.53       662



C:\Users\perry\anaconda3\envs\Py310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\perry\anaconda3\envs\Py310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\perry\anaconda3\envs\Py310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
